## Plots

The Hail plot module allows for easy plotting of data. This notebook contains examples of how to use the plotting functions in this module, many of which can also be found in the first tutorial.

In [ ]:
import hail as hl
hl.init()

from bokeh.io import show, output_notebook
output_notebook()

In [ ]:
hl.utils.get_1kg('data/')
ds = hl.read_matrix_table('data/1kg.mt')
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))
ds = ds.annotate_cols(**table[ds.s])
ds = hl.sample_qc(ds)

ds.describe()

## Histogram

The `histogram()` method takes as an argument an aggregated hist expression, as well as optional arguments for the legend and title of the plot.

In [ ]:
dp_hist = ds.aggregate_entries(hl.expr.aggregators.hist(ds.DP, 0, 30, 30))
p = hl.plot.histogram(dp_hist, legend='DP', title='DP Histogram')
show(p)

This method, like all Hail plotting methods, also allows us to pass in fields of our data set directly. Choosing not to specify the `range` and `bins` arguments would result in a range being computed based on the largest and smallest values in the dataset and a default bins value of 50.

In [ ]:
p = hl.plot.histogram(ds.DP, range=(0, 30), bins=30)
show(p)

## Cumulative Histogram

The `cumulative_histogram()` method works in a similar way to `histogram()`.

In [ ]:
p = hl.plot.cumulative_histogram(ds.DP, range=(0,30), bins=30)
show(p)

## Scatter

The `scatter()` method can also take in either Python types or Hail fields as arguments for x and y.

In [ ]:
p = hl.plot.scatter(ds.sample_qc.dp_mean, ds.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
show(p)

We can also pass in a Hail field as a `label` argument, which determines how to color the data points.

In [ ]:
ds = ds.filter_cols((ds.sample_qc.dp_mean >= 4) & (ds.sample_qc.call_rate >= 0.97))
ab = ds.AD[1] / hl.sum(ds.AD)
filter_condition_ab = ((ds.GT.is_hom_ref() & (ab <= 0.1)) |
                        (ds.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (ds.GT.is_hom_var() & (ab >= 0.9)))
ds = ds.filter_entries(filter_condition_ab)
ds = hl.variant_qc(ds).cache()
common_ds = ds.filter_rows(ds.variant_qc.AF[1] > 0.01)
gwas = hl.linear_regression(y=common_ds.CaffeineConsumption, x=common_ds.GT.n_alt_alleles())
pca_eigenvalues, pca_scores, _ = hl.hwe_normalized_pca(common_ds.GT)

In [ ]:
p = hl.plot.scatter(pca_scores.scores[0], pca_scores.scores[1],
                    label=common_ds.cols()[pca_scores.s].SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

## Q-Q (Quantile-Quantile)

The `qq()` function requires either a Python type or a Hail field containing p-values to be plotted.

In [ ]:
p = hl.plot.qq(gwas.linreg.p_value)
show(p)

## Manhattan

The `manhattan()` function requires a Hail field containing p-values.

In [ ]:
p = hl.plot.manhattan(gwas.linreg.p_value)
show(p)

We can also pass in a dictionary of fields that we would like to show up as we hover over a data point.

In [ ]:
hover_fields = dict([('alleles', gwas.alleles)])
p = hl.plot.manhattan(gwas.linreg.p_value, hover_fields=hover_fields)
show(p)